In [116]:
import pandas as pd

### Load Dataset

In [117]:
train = pd.read_csv("train.csv", index_col = "PassengerId")
print(train.shape)
train.head()

(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [118]:
test = pd.read_csv("test.csv", index_col = "PassengerId")
print(test.shape)
test.head()

(418, 10)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Preprocessing

### Encode Sex

In [119]:
train["Sex_encode"] = train["Sex"].replace("female", 1).replace("male", 0)
print(train.shape)
train[["Sex", "Sex_encode"]].head()

(891, 12)


,Sex,Sex_encode
PassengerId,,
1,male,0
2,female,1
3,female,1
4,female,1
5,male,0


In [120]:
test["Sex_encode"] = test["Sex"].replace("female", 1).replace("male", 0)
print(test.shape)
test[["Sex", "Sex_encode"]].head()

(418, 11)


,Sex,Sex_encode
PassengerId,,
892,male,0
893,female,1
894,male,0
895,male,0
896,female,1


### Fill in missing Fare

In [121]:
test[test["Fare"].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_encode
PassengerId,,,,,,,,,,,
1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,0


In [122]:
test["Fare"] = test["Fare"].fillna(0)   # 418개 중에 하나만 비어있어서 굳이 평균값을 구해서 넣을 필요까진 없고 그냥 0으로 치환

### Encode Embarked

In [123]:
# C == 0, S == 1, Q == 2
# 2 * S == Q?
# S + S == Q?

# 위와 같은 문제를 방지하기 위해 분류할 게 3개 이상이면 아래와 같이 구분한다.

# One Hot Encoding
# True == 1, False == 0
# C == [1, 0, 0]
# S == [0, 1, 0]
# Q == [0, 0, 1]
# 또는
# C == [True, False, False]
# S == [False, True, False]
# Q == [False, False, True]

train["Embarked_C"] = train["Embarked"] == "C"
train["Embarked_S"] = train["Embarked"] == "S"
train["Embarked_Q"] = train["Embarked"] == "Q"

print(train.shape)
train[["Embarked", "Embarked_C", "Embarked_S", "Embarked_Q"]].head()

(891, 15)


,Embarked,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,
1,S,False,True,False
2,C,True,False,False
3,S,False,True,False
4,S,False,True,False
5,S,False,True,False


In [124]:
test["Embarked_C"] = test["Embarked"] == "C"
test["Embarked_S"] = test["Embarked"] == "S"
test["Embarked_Q"] = test["Embarked"] == "Q"

print(test.shape)
test[["Embarked", "Embarked_C", "Embarked_S", "Embarked_Q"]].head()

(418, 14)


,Embarked,Embarked_C,Embarked_S,Embarked_Q
PassengerId,,,,
892,Q,False,False,True
893,S,False,True,False
894,Q,False,False,True
895,S,False,True,False
896,S,False,True,False


In [125]:
mean_age_by_pclass = train.groupby("Pclass")["Age"].mean()

In [126]:
train["Age(fill)"] = train["Age"]
train.loc[(train["Pclass"] == 1) & (train["Age"].isnull()), "Age(fill)"] = mean_age_by_pclass.loc[1]
train.loc[(train["Pclass"] == 2) & (train["Age"].isnull()), "Age(fill)"] = mean_age_by_pclass.loc[2]
train.loc[(train["Pclass"] == 3) & (train["Age"].isnull()), "Age(fill)"] = mean_age_by_pclass.loc[3]
train[["Pclass","Age", "Age(fill)"]].head(10)

,Pclass,Age,Age(fill)
PassengerId,,,
1,3,22.0,22.00000
2,1,38.0,38.00000
3,3,26.0,26.00000
4,1,35.0,35.00000
5,3,35.0,35.00000
6,3,NaN,25.14062
7,1,54.0,54.00000
8,3,2.0,2.00000
9,3,27.0,27.00000


In [127]:
test["Age(fill)"] = test["Age"]
test.loc[(test["Pclass"] == 1) & (test["Age"].isnull()), "Age(fill)"] = mean_age_by_pclass.loc[1]
test.loc[(test["Pclass"] == 2) & (test["Age"].isnull()), "Age(fill)"] = mean_age_by_pclass.loc[2]
test.loc[(test["Pclass"] == 3) & (test["Age"].isnull()), "Age(fill)"] = mean_age_by_pclass.loc[3]
test[["Pclass","Age", "Age(fill)"]].head(30)

,Pclass,Age,Age(fill)
PassengerId,,,
892,3,34.5,34.500000
893,3,47.0,47.000000
894,2,62.0,62.000000
895,3,27.0,27.000000
896,3,22.0,22.000000
897,3,14.0,14.000000
898,3,30.0,30.000000
899,2,26.0,26.000000
900,3,18.0,18.000000


In [151]:
# 나이가 15세 미만인 승객을 색인한 뒤, AgeType이라는 새로운 컬럼에 "Young"이라는 값을 넣습니다.
train.loc[train["Age"] < 15, "AgeType"] = "Young"

# 비슷하게 나이가 15세 이상 30세 미만인 승객의 AgeType에는 "Medium"이라는 값을 넣습니다.
train.loc[(train["Age"] >= 15) & (train["Age"] < 30), "AgeType"] = "Medium"

# 비슷하겍 30세 이상인 승객의 AgeType에는 "Old"이라는 값을 넣습니다.
train.loc[train["Age"] >= 30, "AgeType"] = "Old"

# train 변수에 할당된 데이터의 행렬 사이즈를 출력합니다.
# 출력은 (row, column) 으로 표시됩니다.
print(train.shape)

# 나이(Age) 컬럼과 AgeType 컬럼을 출력하여 비교합니다.
train[["Age", "AgeType"]].head(10)

(891, 19)


,Age,AgeType
PassengerId,,
1,22.0,Medium
2,38.0,Old
3,26.0,Medium
4,35.0,Old
5,35.0,Old
6,NaN,NaN
7,54.0,Old
8,2.0,Young
9,27.0,Medium


In [153]:
test.loc[test["Age"] < 15, "AgeType"] = "Young"
test.loc[(test["Age"] >= 15) & (test["Age"] < 30), "AgeType"] = "Medium"
test.loc[test["Age"] >= 30, "AgeType"] = "Old"
print(test.shape)

test[["Age", "AgeType"]].head(10)

(418, 18)


,Age,AgeType
PassengerId,,
892,34.5,Old
893,47.0,Old
894,62.0,Old
895,27.0,Medium
896,22.0,Medium
897,14.0,Young
898,30.0,Old
899,26.0,Medium
900,18.0,Medium


In [154]:
train["Age_Y"] = train["AgeType"] == "Young"
train["Age_M"] = train["AgeType"] == "Medium"
train["Age_O"] = train["AgeType"] == "Old"

print(train.shape)
train[["Age", "Age_Y", "Age_M", "Age_O"]].head()

(891, 22)


,Age,Age_Y,Age_M,Age_O
PassengerId,,,,
1,22.0,False,True,False
2,38.0,False,False,True
3,26.0,False,True,False
4,35.0,False,False,True
5,35.0,False,False,True


In [155]:
test["Age_Y"] = test["AgeType"] == "Young"
test["Age_M"] = test["AgeType"] == "Medium"
test["Age_O"] = test["AgeType"] == "Old"

print(test.shape)
test[["Age", "Age_Y", "Age_M", "Age_O"]].head()

(418, 21)


,Age,Age_Y,Age_M,Age_O
PassengerId,,,,
892,34.5,False,False,True
893,47.0,False,False,True
894,62.0,False,False,True
895,27.0,False,True,False
896,22.0,False,True,False


In [128]:
train["Single"] = (train["SibSp"] == 0) & (train["Parch"] == 0)
train[["SibSp", "Parch", "Single"]].head()

,SibSp,Parch,Single
PassengerId,,,
1,1,0,False
2,1,0,False
3,0,0,True
4,1,0,False
5,0,0,True


In [129]:
test["Single"] = (test["SibSp"] == 0) & (test["Parch"] == 0)
test[["SibSp", "Parch", "Single"]].head()

,SibSp,Parch,Single
PassengerId,,,
892,0,0,True
893,1,0,False
894,0,0,True
895,0,0,True
896,1,1,False


In [130]:
train["FamilySize"] = train["SibSp"] + train["Parch"] + 1
train[["SibSp", "Parch", "FamilySize"]].head(10)

,SibSp,Parch,FamilySize
PassengerId,,,
1,1,0,2
2,1,0,2
3,0,0,1
4,1,0,2
5,0,0,1
6,0,0,1
7,0,0,1
8,3,1,5
9,0,2,3


In [141]:
test["FamilySize"] = test["SibSp"] + test["Parch"] + 1
test[["SibSp", "Parch", "FamilySize"]].head(10)

,SibSp,Parch,FamilySize
PassengerId,,,
892,0,0,1
893,1,0,2
894,0,0,1
895,0,0,1
896,1,1,3
897,0,0,1
898,0,0,1
899,1,1,3
900,0,0,1


## Train

In [199]:
# Feature - Pclass, Sex, Fare
# Label - Survived
feature_names = ["Pclass", "Sex_encode", "Fare", "Embarked_C", "Embarked_S", "Embarked_Q", "Age_Y", "Age_M", "Age_O", "Single", "FamilySize"]
feature_names

['Pclass',
 'Sex_encode',
 'Fare',
 'Embarked_C',
 'Embarked_S',
 'Embarked_Q',
 'Age_Y',
 'Age_M',
 'Age_O',
 'Single',
 'FamilySize']

In [200]:
X_train = train[feature_names]
print(X_train.shape)
X_train.head()

(891, 11)


,Pclass,Sex_encode,Fare,Embarked_C,Embarked_S,Embarked_Q,Age_Y,Age_M,Age_O,Single,FamilySize
PassengerId,,,,,,,,,,,
1,3,0,7.2500,False,True,False,False,True,False,False,2
2,1,1,71.2833,True,False,False,False,False,True,False,2
3,3,1,7.9250,False,True,False,False,True,False,True,1
4,1,1,53.1000,False,True,False,False,False,True,False,2
5,3,0,8.0500,False,True,False,False,False,True,True,1


In [201]:
X_test = test[feature_names]
print(X_test.shape)
X_test.head()

(418, 11)


,Pclass,Sex_encode,Fare,Embarked_C,Embarked_S,Embarked_Q,Age_Y,Age_M,Age_O,Single,FamilySize
PassengerId,,,,,,,,,,,
892,3,0,7.8292,False,False,True,False,False,True,True,1
893,3,1,7.0000,False,True,False,False,False,True,False,2
894,2,0,9.6875,False,False,True,False,False,True,True,1
895,3,0,8.6625,False,True,False,False,True,False,True,1
896,3,1,12.2875,False,True,False,False,True,False,False,3


In [202]:
label_name = "Survived"
Y_train = train[label_name]
print(Y_train.shape)
Y_train.head()

(891,)


PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Survived, dtype: int64

### Use Decision Tree

In [223]:
# scikit-learn == sklearn
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth = 8)
model

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [224]:
# 1) fit(train) -> 2) predict(test)
model.fit(X_train, Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [225]:
prediction = model.predict(X_test)
print(prediction.shape)
prediction[0:5]

(418,)


array([0, 0, 0, 0, 1])

## Submit

In [226]:
submit = pd.read_csv("gender_submission.csv", index_col = "PassengerId")
submit["Survived"] = prediction
print(submit.shape)
submit.head()

(418, 1)


,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [227]:
submit.to_csv("decision-tree2.csv")